# Exercises

Consider the [MIT-BIH Arrhythmia](https://physionet.org/content/mitdb/1.0.0/) dataset we used in the previous notebook. Now, if you want, you can implement different types of neural networks to solve the classification problem at hand.

## Libraries

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import random
from google.colab import drive
from scipy.signal import resample
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, classification_report

## Utilities

Here, some utility functions are defined. In particular, `train` and `test` are the functions used to train and test models, respectively.

In [ ]:
# Training function.
def train(epoch, model, loader, criterion, optimizer, device='cpu'):
    l = 0
    for data in tqdm(loader, desc=f'Epoch {epoch+1:03d}'):
        x = data[0].to(device)
        y = data[1].to(device)
        out = model(x)
        loss = criterion(out, y)
        l += loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return l

# Test function.
def test(model, loader, criterion, device='cpu'):
    l = 0
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in loader:
            x = data[0].to(device)
            y = data[1].to(device)
            out = model(x)
            l += criterion(out, y)
            _, pred = torch.max(out.data, 1)
            total += y.size(0)
            correct += (pred == y).sum().item()
            y_true += y.tolist()
            y_pred += pred.tolist()
    return l, correct / total, f1_score(y_true, y_pred, average='macro'), y_true, y_pred

This code cell defines three classes (`Stretch`, `Amplify`, and `Augment`) for randomly augmenting input signals.

In [ ]:
# Randomly stretches the signal.
class Stretch:
    def __init__(self):
        pass

    def __call__(self, x):
        n = x.shape[0]
        l = int(n * (1 + (random.random() - 0.5) / 3))
        y = resample(x, l)
        if l < n:
            y_ = np.zeros(shape=(n,))
            y_[:l] = y
        else:
            y_ = y[:n]
        return y_

    def __repr__(self):
        return 'Stretch'

# Randomly amplifies the signal.
class Amplify:
    def __init__(self):
        pass

    def __call__(self, x):
        alpha = (random.random() - 0.5)
        factor = -alpha * x + (1 + alpha)
        return x * factor

    def __repr__(self):
        return 'Amplify'

# Randomly augments the input signal.
class Augment:
    def __init__(self, aug_list, verbose=False):
        self.aug_list = aug_list
        self.verbose = verbose

    def __call__(self, x):
        augs = ''
        for i, aug in enumerate(self.aug_list):
            if np.random.binomial(1, 0.5) == 1:
                x = aug(x)
                augs += f'{aug}, ' if i < len(self.aug_list) - 1 else f'{aug}'
        if not self.verbose:
            return x
        return x, augs

The `CustomDataset` class encapsulates the functionality required to prepare custom data for training deep learning models in PyTorch.

In [ ]:
# Custom dataset, as we discussed in the first lecture (PyTorch Basics).
class CustomDataset(Dataset):

    # Stores the data.
    def __init__(self, x, y=None, transforms=None):
        super().__init__()
        self.x = x
        self.y = y
        self.transforms = transforms

    # Returns the length of the dataset.
    def __len__(self):
        return self.x.shape[0]

    # Returns a (x, y) pair from the dataset.
    def __getitem__(self, idx):
        x = self.x.iloc[idx, :]
        if self.transforms is not None:
            x = self.transforms(x)
        x = torch.tensor(x).float().unsqueeze(-1)
        y = torch.tensor([self.y.iloc[idx]]).type(torch.LongTensor).squeeze()
        return x, y

## Data

To run the following cells, you should first download the dataset and then upload the two files, `mitbih_train.csv` and `mitbih_test.csv`, here on Google Colab. Otherwise, you can upload the files on Google Drive and access them by connecting Colab to Drive.

In [ ]:
# Connect Google Drive.
drive.mount('/content/drive')

# Read train and test data.
df_train = pd.read_csv('drive/MyDrive/mitbih_train.csv', header=None)
df_test = pd.read_csv('drive/MyDrive/mitbih_test.csv', header=None)

# Training dataframe.
df_train

We define the batch size and then create the `Dataset` objects.

In [ ]:
# Batch size.
batch_size = 64

# New Augment object.
augment = Augment([Amplify(), Stretch()])

# Dataset objects.
train_dataset = ...
test_dataset = ...

We then create the `WeightedRandomSampler` object to perform data augmentation, and define the `DataLoader` objects.



In [ ]:
# List of training labels.
train_targets = ...

# Computing class weights based on class frequency.
cls_weights = ...

# Resulting array of weights.
weights = ...

# Weighted random sampler. Used to consider copies of minority classes.
sampler = ...

# DataLoader objects.
train_loader = ...
test_loader = ...

## Model Definition

First, we select the device.

In [ ]:
# Device configuration.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Then, we define the model.

### $m_1$: Feature Extraction $+$ MLP

Define an MLP that takes as input a tensor of statistical features extracted from each input batch and returns the classification scores.

You can consider different types of statistical features: [mean](https://pytorch.org/docs/stable/generated/torch.mean.html), [standard deviation](https://pytorch.org/docs/stable/generated/torch.std.html), [max](https://pytorch.org/docs/stable/generated/torch.max.html), [min](https://pytorch.org/docs/stable/generated/torch.min.html), etc.

In [ ]:
# Model hyperparameters.
...

# Refer to the 0-pytorch-basics.ipynb notebook.
class MLP(nn.Module):
    def __init__(self, ...):
        super(MLP, self).__init__()
        # Define multiple nn.Linear layers.
        ...

    def forward(self, x):
        # 1. Extract statistical features from the input.
        # 2. Forward pass through nn.Linear layers.
        ...

# New model.
model = MLP(...).to(device)

### $m_2$: 1D-Convolutional Backbone $+$ Classification Head

Define a 1D-CNN composed of a 1D-convolutional backbone that extracts features from the input signal, and a final classification head used to produce the classification scores.

Pay attention to the input and output shape of each layer, as discussed earlier in the course.

In [ ]:
# Model hyperparameters.
...

# Refer to the 1-cnns.ipynb notebook.
class CNN(nn.Module):
    def __init__(self, ...):
        super(CNN, self).__init__()
        # Define multiple nn.Conv1d layers.
        # Define a single nn.Linear which acts as the final classification head.
        ...

    def forward(self, x):
        # 1. Forward pass through nn.Conv1d layers.
        # 2. Forward pass through the nn.Linear layer.
        ...

# New model.
model = CNN(...).to(device)

### $m_3$: 1D-Convolutional Feature Extractor $+$ LSTM Layer $+$ Classification Head

Define a model composed of one or more 1D-convolutional layers, an LSTM layer, and a final classification head used to produce the classification scores.

Pay attention to the input and output shape of each layer, as discussed earlier in the course.

In [ ]:
# Model hyperparameters.
...

# Refer to the 1-cnns.ipynb, 2-rnns.ipynb notebooks.
class CNN_LSTM(nn.Module):
    def __init__(self, ...):
        super(CNN_LSTM, self).__init__()
        # Define one or more nn.Conv1d layers to extract features.
        # Define a single LSTM layer.
        # Define a single nn.Linear which acts as the final classification head.
        ...

    def forward(self, x):
        # 1. Forward pass through nn.Conv1d layers.
        # 2. Forward pass through the nn.LSTM layer.
        # 3. Forward pass through the nn.Linear layer.
        ...

# New model.
model = CNN_LSTM(...).to(device)

## Training and Test

We define few other parameters:
- The duration of training (`num_epochs`).
- The pace at which our model learns (`learning_rate`).

In [ ]:
# Training hyperparameters.
num_epochs = 10
learning_rate = 0.0001

Lastly, we train and test the model.

### $m_1$: Feature Extraction $+$ MLP

In [ ]:
# Loss and optimizer.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training and test.
for epoch in range(num_epochs):
    train_loss = train(epoch, model, train_loader, criterion, optimizer, device)
    test_loss, test_acc, test_f1, y_true, y_pred = test(model, test_loader, criterion, device)
    print(f'Epoch {epoch+1:03d}: training loss {train_loss:.4f}, test loss {test_loss:.4f}, test acc {test_acc:.4f}, test f1 {test_f1:.4f}')

In [ ]:
print(classification_report(y_true, y_pred, zero_division=0))

### $m_2$: 1D-Convolutional Backbone $+$ Classification Head

In [ ]:
# Loss and optimizer.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training and test.
for epoch in range(num_epochs):
    train_loss = train(epoch, model, train_loader, criterion, optimizer, device)
    test_loss, test_acc, test_f1, y_true, y_pred = test(model, test_loader, criterion, device)
    print(f'Epoch {epoch+1:03d}: training loss {train_loss:.4f}, test loss {test_loss:.4f}, test acc {test_acc:.4f}, test f1 {test_f1:.4f}')

In [ ]:
print(classification_report(y_true, y_pred, zero_division=0))

### $m_3$: 1D-Convolutional Feature Extractor $+$ LSTM Layer $+$ Classification Head

In [ ]:
# Loss and optimizer.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training and test.
for epoch in range(num_epochs):
    train_loss = train(epoch, model, train_loader, criterion, optimizer, device)
    test_loss, test_acc, test_f1, y_true, y_pred = test(model, test_loader, criterion, device)
    print(f'Epoch {epoch+1:03d}: training loss {train_loss:.4f}, test loss {test_loss:.4f}, test acc {test_acc:.4f}, test f1 {test_f1:.4f}')

In [ ]:
print(classification_report(y_true, y_pred, zero_division=0))